In [28]:
import sqlalchemy
import pandas as pd
from datetime import datetime, timedelta
import numpy as np

In [29]:
engstr = ""
engine = sqlalchemy.create_engine(engstr)

In [30]:
def get_dataframe_from_sql(eng, sql_stmt):
    df = pd.read_sql(sql_stmt, eng)
    return df.copy()

In [31]:
df = get_dataframe_from_sql(engine, 'SELECT * FROM etheur_ohlc_5;')
df

,time_end,open,high,low,close,volume
0,2022-01-02 12:05:00,3296.41,3301.51,3296.41,3301.09,8.615494
1,2022-01-02 12:10:00,3300.35,3300.35,3299.53,3299.54,3.631679
2,2022-01-02 12:15:00,3299.54,3299.55,3298.68,3298.68,4.276312
3,2022-01-02 12:20:00,3298.68,3302.52,3298.68,3301.20,8.266984
4,2022-01-02 12:25:00,3301.22,3302.41,3301.21,3301.85,5.305136
...,...,...,...,...,...,...
188,2022-01-02 22:50:00,3365.03,3365.59,3359.73,3359.90,5.799117
189,2022-01-02 22:50:00,3365.03,3365.59,3359.71,3359.72,5.883703
190,2022-01-02 22:50:00,3365.03,3365.59,3359.68,3359.68,6.035203
191,2022-01-02 22:50:00,3365.03,3365.59,3359.46,3359.46,6.083361


In [32]:
lastEntry = df.iloc[-1:]
lastEntry

,time_end,open,high,low,close,volume
192,2022-01-02 22:50:00,3365.03,3365.59,3359.46,3359.76,6.241266


In [33]:
test1 = df["time_end"][-1:].values[0]

In [34]:
test2 = np.datetime64(datetime.now())

In [35]:
print(test1, test2)
print(test2 < test1)
print(test2 > (test1 - np.timedelta64(5,"m")))

2022-01-02T22:50:00.000000000 2022-01-02T22:47:51.268266
True
True


In [36]:
import krakenex
k = krakenex.API()

In [37]:
k.query_public("Time")

{'error': [],
 'result': {'unixtime': 1641160075,
  'rfc1123': 'Sun,  2 Jan 22 21:47:55 +0000'}}

In [38]:
k.load_key(r'C:\Users\NickK\git\KryptoTrading\kraken.key')

In [46]:
balance = k.query_private("Balance")
float(balance['result']['ZEUR'])

149.2771

In [48]:
k.query_private("TradesHistory")

{'error': [],
 'result': {'count': 2,
  'trades': {'TG2DGL-4NMB3-IYOCUH': {'ordertxid': 'OIZVQ2-UZWDH-E5ATLL',
    'postxid': 'TKH2SE-M7IF5-CFI7LT',
    'pair': 'XETHZEUR',
    'time': 1641161174.883178,
    'type': 'sell',
    'ordertype': 'market',
    'price': '3359.24000',
    'cost': '134.97910',
    'fee': '0.35095',
    'vol': '0.04018144',
    'margin': '0.00000',
    'misc': ''},
   'TJWI3H-AWQEF-IRAWW3': {'ordertxid': 'OZKVL6-SYWP5-AESBPO',
    'postxid': 'TKH2SE-M7IF5-CFI7LT',
    'pair': 'XETHZEUR',
    'time': 1641160089.8840225,
    'type': 'buy',
    'ordertype': 'market',
    'price': '3359.76000',
    'cost': '134.99999',
    'fee': '0.35100',
    'vol': '0.04018144',
    'margin': '0.00000',
    'misc': ''}}}}

In [47]:
balance = float(balance['result']['ZEUR'])
ETHval = lastEntry['close'].values[0]
tradeVol = 0.95*(balance/ETHval)
tradeVol

0.04220933786937161

In [42]:
k.query_private('AddOrder',
                        {'pair': 'ETHEUR',
                         'type': 'buy',
                         'ordertype': 'market',
                         'volume': tradeVol,
                         })

{'error': [],
 'result': {'txid': ['OZKVL6-SYWP5-AESBPO'],
  'descr': {'order': 'buy 0.04018144 ETHEUR @ market'}}}

In [43]:
k.query_private('AddOrder',
                        {'pair': 'ETHEUR',
                         'type': 'sell',
                         'ordertype': 'market',
                         'volume': tradeVol,
                         })

{'error': [],
 'result': {'txid': ['OIZVQ2-UZWDH-E5ATLL'],
  'descr': {'order': 'sell 0.04018144 ETHEUR @ market'}}}

In [28]:
k.query_private('CancelAll')

{'error': [], 'result': {'count': 0}}

In [35]:
k.query_private('DepositMethods',
                   {'asset': 'ETH'})

{'error': [],
 'result': [{'method': 'Ether (Hex)', 'limit': False, 'gen-address': True},
  {'method': 'Ethereum (ERC20)', 'limit': False, 'gen-address': True}]}

In [42]:
k.query_private('DepositAddresses',
                   {'asset': 'ETH',
                    'method':'Ethereum (ERC20)'})

{'error': [],
 'result': [{'address': '0x282ee2e5f75c030f92d41cfed81bce3cc4a33e19',
   'expiretm': '0',
   'new': True}]}